In [90]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

from scipy.io import loadmat
from dotmap import DotMap
from scipy.stats import zscore

saved = DotMap(loadmat('/Users/yfukuzaw/tmp/mt-lbi11.mat'))
sids = saved.sids.ravel()
dataset = saved.dataset
clusters = saved.clusters
meas = zscore(dataset)
labels = saved.labels
haslabel_ind = np.where(labels!='                              ')[0]

labels = labels[haslabel_ind]
dataset = dataset[haslabel_ind, :]
sids = sids[haslabel_ind]
clusters = clusters[haslabel_ind, :]

unique_labels = np.unique(labels)
traces = []
for label in unique_labels:
    ind = np.where(labels==label)
    x = clusters[ind, 0].ravel();
    y = clusters[ind, 1].ravel();
    z = clusters[ind, 2].ravel();
    ids = sids[ind];
    
    trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name=label.strip(),
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                width=0.5
            ),
            opacity=1
        ),
        text=ids
    )
    traces.append(trace1)
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')